In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
import nltk
import os
import pickle
from nltk import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import string
import re, itertools

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import ParameterGrid
from sklearn.pipeline import Pipeline

SQL CODE:

SELECT *
FROM Posts
WHERE
    PostTypeId = 1 AND
    Score >=0 AND
    CreationDate >= '20180101' AND
    Tags <> 0
    

In [2]:
#dataloading

#To copy paste in the list of parameters below to limit the dataset size
postsdf = pd.read_csv('posts with tags.csv', nrows = 50, index_col=0, parse_dates=['CreationDate', 'DeletionDate', 'LastEditDate', 'LastActivityDate', 'ClosedDate'])

#complete loading
postsdf = pd.read_csv('posts with tags.csv', index_col=0, parse_dates=['CreationDate', 'DeletionDate', 'LastEditDate', 'LastActivityDate', 'ClosedDate'])
postsdf.head()

,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
Id,,,,,,,,,,,,,,,,,,,,,
48046688,1,NaN,NaN,2018-01-01 01:43:05,NaT,0,32,<p>Hi I was trying to install a python package...,9159552.0,NaN,...,NaN,2018-01-01 02:50:41,2018-01-01 02:50:41,tried to install a pythonn package but encount...,<python><gcc>,0,0,NaN,NaT,NaN
48046690,1,NaN,NaN,2018-01-01 01:44:46,NaT,0,39,<p>I am trying to make a chrome extension that...,9159517.0,NaN,...,NaN,2018-01-01 16:59:50,2018-01-01 16:59:50,Update chrome extension popup text,<javascript><html><google-chrome-extension>,0,3,NaN,NaT,NaN
48046378,1,NaN,NaN,2018-01-01 00:00:29,NaT,0,39,<p>Calling a <em>dll</em> from Python using <e...,8276973.0,NaN,...,NaN,2018-01-01 03:16:54,2018-01-03 08:21:23,Python ctypes: how to pass ctypes array to DLL?,<python><ctypes>,2,0,NaN,NaT,NaN
48046379,1,NaN,NaN,2018-01-01 00:00:32,NaT,0,390,<p>I have a Django app with which users can cr...,1745384.0,NaN,...,NaN,2018-01-01 03:18:33,2018-01-11 23:14:31,nginx gunicorn 502 bad gateway: upstream prema...,<python><django><nginx>,1,0,1.0,NaT,NaN
48046382,1,NaN,NaN,2018-01-01 00:00:52,NaT,0,44,<p>I'm trying to implement hittest that is use...,8794406.0,NaN,...,NaN,NaT,2018-01-01 00:00:52,Implementing hit test,<c++><opencv><3d><computer-vision><arkit>,0,0,NaN,NaT,NaN


In [3]:
#List of columns availabale
postsdf.columns.values

array(['PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'DeletionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
       'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName',
       'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
       'CommentCount', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate'], dtype=object)

In [4]:
#Tags parsing on dataset
postsdf['PTags'] = postsdf['Tags'].apply(lambda x: [tag.name for tag in BeautifulSoup(x, 'html.parser').find_all()])
postsdf['LenTags'] = postsdf['PTags'].apply(lambda x: len(x))
postsdf[['PTags', 'Tags', 'LenTags']].sort_values('LenTags', ascending = False).head(10)

,PTags,Tags,LenTags
Id,,,
48094601,"[list, function, sharepoint-2010, calculated-c...",<list><function><sharepoint-2010><calculated-c...,5
48180277,"[java, spring, spring-boot, cron, spring-sched...",<java><spring><spring-boot><cron><spring-sched...,5
48079684,"[docker, docker-compose, docker-swarm, docker-...",<docker><docker-compose><docker-swarm><docker-...,5
48126497,"[ios, swift, video, rotation, avfoundation]",<ios><swift><video><rotation><avfoundation>,5
48080749,"[c++, boost, number-formatting, standard-libra...",<c++><boost><number-formatting><standard-libra...,5
48080750,"[java, android, sphinx4, mfcc, pocketsphinx-an...",<java><android><sphinx4><mfcc><pocketsphinx-an...,5
48079288,"[javascript, jquery, html, youtube-api, youtub...",<javascript><jquery><html><youtube-api><youtub...,5
48144803,"[matlab, user-interface, plot, axes, markers]",<matlab><user-interface><plot><axes><markers>,5
48080206,"[javascript, firebase, object, firebase-databa...",<javascript><firebase><object><firebase-databa...,5


In [5]:
#What are the tags in the HTML body ?
postsdf['TagsSet'] = postsdf['Body'].apply(lambda x: set([tag.name for tag in BeautifulSoup(x, 'lxml').find_all()]))
postsdf['TagsSetLen'] = postsdf['TagsSet'].apply(lambda x: len(x))
print(postsdf['TagsSetLen'].describe())
#print(postsdf['TagsSet'].str.join(sep='*').str.get_dummies(sep='*').describe())
postsdf[['TagsSet', 'Body', 'TagsSetLen']].sort_values('TagsSetLen', ascending = False).head(20)

count    50000.000000
mean         5.413480
std          1.407265
min          3.000000
25%          5.000000
50%          5.000000
75%          6.000000
max         15.000000
Name: TagsSetLen, dtype: float64


,TagsSet,Body,TagsSetLen
Id,,,
48176897,"{html, p, h3, pre, code, li, ol, h2, strong, i...","<p>I am pretty new to knockoutJS, have complet...",15
48059195,"{html, p, pre, code, li, ol, img, strong, div,...","<p>I have been <a href=""https://github.com/kau...",14
48118901,"{html, p, h3, code, li, strong, img, body, em,...",<p>I have two services running on Azure :</p>\...,14
48171199,"{html, p, pre, code, blockquote, li, h2, ol, s...",<p>I'm build the backend for a web app. When a...,14
48173979,"{html, p, pre, code, h2, ol, li, blockquote, s...",<h2>Find length of line 300* slower</h2>\n\n<p...,14
48116359,"{html, p, pre, code, li, ol, strong, body, br,...",<p>Although it's much more complex than I'm ab...,14
48080865,"{html, p, pre, code, blockquote, h2, ol, li, s...",<p>I try to analyze one module of a multi modu...,13
48144185,"{html, p, pre, code, li, ol, h2, img, strong, ...",<p>I have the following custom view <code>Dial...,13
48176556,"{html, p, pre, code, blockquote, ol, li, stron...",<p><strong>Context/Background</strong>: I have...,13


https://www.w3schools.com/TAGs/

In [6]:
#List of all the HTML tags in the dataset
postsdf['TagsSet'].str.join(sep='*').str.get_dummies(sep='*').columns.values

array(['a', 'b', 'blockquote', 'body', 'br', 'code', 'dd', 'div', 'dl',
       'dt', 'em', 'h1', 'h2', 'h3', 'hr', 'html', 'i', 'img', 'kbd', 'li',
       'ol', 'p', 'pre', 's', 'strike', 'strong', 'sub', 'sup', 'ul'], dtype=object)

In [7]:
#Title  - general information
postsdf['LenTitle'] = postsdf['Title'].apply(lambda x:len(x.split(' ')))
postsdf[['LenTitle', 'Title']].sort_values('LenTitle', ascending = False).head(20)

,LenTitle,Title
Id,,
48071275,32,TV channel in C# and I'm using Vlc API. When t...
48158843,30,If I have a function that prints as many times...
48109943,30,Snakemake only re-runs jobs if one of the inpu...
48159052,29,I'm trying to write a script to search a folde...
48050389,29,How do I build apk for android API 21 using Io...
48160040,29,"I'm working on Euler 12 , the code i have seem..."
48059619,29,"I am trying to attach pdf,docs &pics from gall..."
48076028,29,I am sending an int object on a socket connect...
48064455,29,When app is not opened and when i try to inser...


In [8]:
#Prototype
#General parsing test

sampleline = 48048147

textsample = postsdf['Body'].loc[sampleline]
titlesample = postsdf['Title'].loc[sampleline]
tagssample = postsdf['Tags'].loc[sampleline]
soupBody = BeautifulSoup(textsample, 'lxml')
soupTags = BeautifulSoup(tagssample, 'html.parser')

In [9]:
#Tags parsing
'|'.join(set([tag.name for tag in soupTags.find_all()]))

'file-io|file|byte-order-mark|c++|ifstream'

In [10]:
#Raw version Body
print(BeautifulSoup(postsdf['Body'].loc[sampleline], 'lxml').prettify())

<html>
 <body>
  <p>
   I have a .txt file called "1.txt" that I want to read in.  Since the file starts with 8 BOM characters, if I do the following:
  </p>
  <pre><code>ifstream fin("1.txt");

string temp = "";

char c = fin.get();

    while (!fin.eof())
    {
        if (c &gt;= ' ' &amp;&amp; c &lt;= 'z')
        {
            temp += c;
        }

        c = fin.get();
    }

    cout &lt;&lt; temp;
</code></pre>
  <p>
   This will print nothing, because of something the BOM is doing.
  </p>
  <p>
   So, I decided to use the fin.ignore() function, in order to ignore the beginning BOM characters of the file.  However, still nothing is being printed.  Here is my complete program:
  </p>
  <pre><code>#include &lt;iostream&gt;
#include &lt;fstream&gt;
#include &lt;string&gt;
#include &lt;istream&gt;

using namespace std;

int main()
{
ifstream fin("1.txt");

if (fin.fail())
{
    cout &lt;&lt; "Fail\n";
}

else
{
    string temp = ""; // Will hold 1.txt's contents.

    fin.ignore(1

In [11]:
#Code parsing
for text in soupBody.find_all('code'):
    print(text.text)

ifstream fin("1.txt");

string temp = "";

char c = fin.get();

    while (!fin.eof())
    {
        if (c >= ' ' && c <= 'z')
        {
            temp += c;
        }

        c = fin.get();
    }

    cout << temp;

#include <iostream>
#include <fstream>
#include <string>
#include <istream>

using namespace std;

int main()
{
ifstream fin("1.txt");

if (fin.fail())
{
    cout << "Fail\n";
}

else
{
    string temp = ""; // Will hold 1.txt's contents.

    fin.ignore(10, ' ');
    // Ignore first 10 chars of the file or stop at the first space char,
    // since the BOM at the beginning is causing problems for fin to read the file.
    // BOM is 8 chars, I wrote 10 to just be safe.

    char c = fin.get();

    while (!fin.eof())
    {
        if (c >= ' ' && c <= 'z') // checks if c stores a standard char.
        {
            temp += c;
        }

        c = fin.get();
    }

    cout << temp;

    // PROBLEM:  No text is printed to the screen from the above command.

    cout <

In [12]:
#Text parsing only with p tags
for text in soupBody.find_all('p'):
    print(text.text)

I have a .txt file called "1.txt" that I want to read in.  Since the file starts with 8 BOM characters, if I do the following:
This will print nothing, because of something the BOM is doing.
So, I decided to use the fin.ignore() function, in order to ignore the beginning BOM characters of the file.  However, still nothing is being printed.  Here is my complete program:
I hypothesize that after the:  ifstream fin("1.txt");  line, it is already too late, since the BOM probably affected things with fin then.  So I need to somehow tell fin to ignore the BOM characters before it reads in the file, but I can't use fin.ignore() since I wouldn't have declared a fin object yet.
Also, I know I can manually delete the BOM from my .txt file, but I'm looking for a solution that only involves me writing a C++ program.  If I have thousands or millions of .txt files, deleting manually is not an option.  Also, I'm not looking to download new software, like Notepad++
Here is all I have in the file  "1.t

In [13]:
#Feature to parse posts and exclude links
def postsparsing(entry):
    toreturn = ''
    soup = BeautifulSoup(entry, 'lxml')
    for bloc in soup.find_all('p'):
        if not bloc.findChildren():
            toreturn += ' ' + bloc.text
        elif bloc.findChildren()[0].name != 'a':
            toreturn += ' ' + bloc.text
    return toreturn

postsparsing(postsdf['Body'].loc[sampleline])

' I have a .txt file called "1.txt" that I want to read in.  Since the file starts with 8 BOM characters, if I do the following: This will print nothing, because of something the BOM is doing. So, I decided to use the fin.ignore() function, in order to ignore the beginning BOM characters of the file.  However, still nothing is being printed.  Here is my complete program: I hypothesize that after the:  ifstream fin("1.txt");  line, it is already too late, since the BOM probably affected things with fin then.  So I need to somehow tell fin to ignore the BOM characters before it reads in the file, but I can\'t use fin.ignore() since I wouldn\'t have declared a fin object yet. Also, I know I can manually delete the BOM from my .txt file, but I\'m looking for a solution that only involves me writing a C++ program.  If I have thousands or millions of .txt files, deleting manually is not an option.  Also, I\'m not looking to download new software, like Notepad++ Here is all I have in the file

In [14]:
#text parsing - dataframe update
postsdf['PText'] = postsdf['Body'].apply(postsparsing)
postsdf['PCode'] = postsdf['Body'].apply(lambda x: ' '.join([text.text for text in BeautifulSoup(x, 'lxml').find_all('code')]))
postsdf[['PText', 'PCode', 'Body']].head()

,PText,PCode,Body
Id,,,
48046688,Hi I was trying to install a python package c...,pip install pysptk\n /usr/bin/ld: cannot find ...,<p>Hi I was trying to install a python package...
48046690,I am trying to make a chrome extension that d...,"<script src=""popup.js""></script>\n<html>\n<hea...",<p>I am trying to make a chrome extension that...
48046378,"Calling a dll from Python using ctypes, I wan...",ReturnVec = ctypes.c_float * len(arrA)\nt = ty...,<p>Calling a <em>dll</em> from Python using <e...
48046379,I have a Django app with which users can crea...,client_max_body_size 100M\n fastcgi_buffers 8 ...,<p>I have a Django app with which users can cr...
48046382,I'm trying to implement hittest that is used ...,"std::pair<bool, double> linePlaneIntersection...",<p>I'm trying to implement hittest that is use...


In [15]:
#Review all the results
postsdf[['PText', 'PCode', 'Body', 'PTags', 'Tags']].head()

,PText,PCode,Body,PTags,Tags
Id,,,,,
48046688,Hi I was trying to install a python package c...,pip install pysptk\n /usr/bin/ld: cannot find ...,<p>Hi I was trying to install a python package...,"[python, gcc]",<python><gcc>
48046690,I am trying to make a chrome extension that d...,"<script src=""popup.js""></script>\n<html>\n<hea...",<p>I am trying to make a chrome extension that...,"[javascript, html, google-chrome-extension]",<javascript><html><google-chrome-extension>
48046378,"Calling a dll from Python using ctypes, I wan...",ReturnVec = ctypes.c_float * len(arrA)\nt = ty...,<p>Calling a <em>dll</em> from Python using <e...,"[python, ctypes]",<python><ctypes>
48046379,I have a Django app with which users can crea...,client_max_body_size 100M\n fastcgi_buffers 8 ...,<p>I have a Django app with which users can cr...,"[python, django, nginx]",<python><django><nginx>
48046382,I'm trying to implement hittest that is used ...,"std::pair<bool, double> linePlaneIntersection...",<p>I'm trying to implement hittest that is use...,"[c++, opencv, computer-vision, arkit]",<c++><opencv><3d><computer-vision><arkit>


In [16]:
#Custom tokenizer
def customtokenizer(entry, negate = True):
    
    #Breaks in sentences
    sentences = sent_tokenize(entry, language='english')
    
    #Breaks in words
    tokenizedsentences = []
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        tokenizedsentences.append(tokens)
    
    #Stem the sentences
    stemmer = SnowballStemmer("english")
    stemmed = []
    for sentence in tokenizedsentences:
        sentencestemmed = [stemmer.stem(word.lower()) for word in sentence]
        stemmed.append(sentencestemmed)
    
    #Applys the negation if needed
    if negate:
        sentiments = []
        for sentence in stemmed:
            tokens = mark_negation(sentence)
            sentiments.append(tokens)
    else:
        sentiments = stemmed
    
    #Cleans the Stop-words
    stopwords = set(nltk.corpus.stopwords.words('english'))
    stopwords.add('...')
    cleanedsw = []
    for sentence in sentiments:
        tokens = [word for word in sentence if (word not in stopwords) and ((word[:-4] not in stopwords) if word.endswith('_NEG') else True)]
        cleanedsw.append(tokens)
    
    #Clean the punctuation
    cleanedpunct = []
    for sentence in cleanedsw:
        tokens = [word for word in sentence if (word not in string.punctuation) and ((word[:-4] not in string.punctuation) if word.endswith('_NEG') else True)]
        cleanedpunct.append(tokens)
        
    #Clean the numbers
    cleanednumber = []
    for sentence in cleanedpunct:
        tokens = [word for word in sentence if not word.isdigit()]
        cleanednumber.append(tokens)
    
    
    #Sentences assembly
    assembly = []
    for sentence in cleanednumber:
        tokens = [word for word in sentence]
        assembly.extend(sentence)

    return ' '.join(assembly)
customtokenizer('''The movie, wasn't that, good. It was terribly bad. Hello, I'm 50% sure that but javascript or c++ sucks! On the other hand python is so much better, I wish i was kiding :-). R is pretty good.''')

"movi n't good_NEG terribl bad hello 'm sure javascript c++ suck hand python much better wish kide r pretti good"

In [17]:
customtokenizer('01')

''

In [18]:
#POST - Result for sample post - original version
postsdf['PText'].loc[sampleline]

' I have a .txt file called "1.txt" that I want to read in.  Since the file starts with 8 BOM characters, if I do the following: This will print nothing, because of something the BOM is doing. So, I decided to use the fin.ignore() function, in order to ignore the beginning BOM characters of the file.  However, still nothing is being printed.  Here is my complete program: I hypothesize that after the:  ifstream fin("1.txt");  line, it is already too late, since the BOM probably affected things with fin then.  So I need to somehow tell fin to ignore the BOM characters before it reads in the file, but I can\'t use fin.ignore() since I wouldn\'t have declared a fin object yet. Also, I know I can manually delete the BOM from my .txt file, but I\'m looking for a solution that only involves me writing a C++ program.  If I have thousands or millions of .txt files, deleting manually is not an option.  Also, I\'m not looking to download new software, like Notepad++ Here is all I have in the file

In [19]:
#POST - Result for sample post - TOKENIZED version
customtokenizer(postsdf['PText'].loc[sampleline])

".txt file call `` 1.txt '' want read sinc file start bom charact follow print noth becaus someth bom decid use fin.ignor function order ignor begin bom charact file howev still noth print complet program hypothes ifstream fin `` 1.txt '' line alreadi late sinc bom probabl affect thing fin need somehow tell fin ignor bom charact befor read file ca n't use_NEG fin.ignor_NEG sinc_NEG would_NEG n't_NEG declar_NEG fin_NEG object_NEG yet_NEG also know manual delet bom .txt file 'm look solut onli involv write c++ program thousand million .txt file delet manual option_NEG also 'm look_NEG download_NEG new_NEG softwar_NEG like_NEG notepad++_NEG file_NEG ``_NEG 1.txt_NEG ''_NEG ðïà¡±á hello site 's format doe n't let_NEG show_NEG actual_NEG file_NEG 15_NEG space_NEG bom_NEG hello_NEG"

In [20]:
#TITLE - Result for sample title - original version
postsdf['Title'].loc[sampleline]

'C++ - Am I using fin.ignore() incorrectly?'

In [21]:
#TITLE - Result for sample title - TOKENIZED version
customtokenizer(postsdf['Title'].loc[sampleline], negate = False)

'c++ use fin.ignor incorrect'

In [22]:
#Code tokenizer creation
text = """
class Foldermanagement():
def __init__(self):
    self.today = invoicemng.gettoday() + 7
       """
def codetokenizer(text):
    separators = string.punctuation + string.whitespace
    separators = [sep for sep in separators if sep != '_']
    separators_re = "|".join(re.escape(x) for x in separators)
    tokens = zip(re.split(separators_re, text), re.findall(separators_re, text))
    flattened = itertools.chain.from_iterable(tokens)
    cleaned = [x for x in flattened if x and not x.isspace()]
    
    #To gather the symbols by 2
    toreturn = []
    skip = False
    for index in range(len(cleaned)-1):
        if cleaned[index] in string.punctuation and cleaned[index+1] in string.punctuation and not skip:
            toreturn.append(cleaned[index]+cleaned[index+1])
            skip = True
        elif skip == True:
            skip = False
        else:
            toreturn.append(cleaned[index])
    
    #Digit cleaning
    toreturn = [word for word in toreturn if not word.isdigit()]
            
    return ' '.join(toreturn)
    
codetokenizer(text)

'class Foldermanagement () : def __init__ ( self ): self . today = invoicemng . gettoday () +'

In [23]:
#CODE - Result for sample code - original version

print(postsdf['PCode'].loc[sampleline])

ifstream fin("1.txt");

string temp = "";

char c = fin.get();

    while (!fin.eof())
    {
        if (c >= ' ' && c <= 'z')
        {
            temp += c;
        }

        c = fin.get();
    }

    cout << temp;
 #include <iostream>
#include <fstream>
#include <string>
#include <istream>

using namespace std;

int main()
{
ifstream fin("1.txt");

if (fin.fail())
{
    cout << "Fail\n";
}

else
{
    string temp = ""; // Will hold 1.txt's contents.

    fin.ignore(10, ' ');
    // Ignore first 10 chars of the file or stop at the first space char,
    // since the BOM at the beginning is causing problems for fin to read the file.
    // BOM is 8 chars, I wrote 10 to just be safe.

    char c = fin.get();

    while (!fin.eof())
    {
        if (c >= ' ' && c <= 'z') // checks if c stores a standard char.
        {
            temp += c;
        }

        c = fin.get();
    }

    cout << temp;

    // PROBLEM:  No text is printed to the screen from the above command.

    cout <

In [24]:
#CODE - Result for sample code - TOKENIZED version
' '.join(codetokenizer(postsdf['PCode'].loc[sampleline]))
codetokenizer(postsdf['PCode'].loc[sampleline])

'ifstream fin (" . txt ") ; string temp =" "; char c = fin . get () ; while (! fin . eof () ){ if ( c >= \'\' && c <= \' z \') { temp += c ;} c = fin . get () ;} cout << temp ;# include < iostream ># include < fstream ># include < string ># include < istream > using namespace std ; int main () { ifstream fin (" . txt ") ; if ( fin . fail () ){ cout << " Fail \\ n "; } else { string temp =" "; // Will hold . txt \' s contents . fin . ignore ( ,\' \') ;/ / Ignore first chars of the file or stop at the first space char ,/ / since the BOM at the beginning is causing problems for fin to read the file ./ / BOM is chars , I wrote to just be safe . char c = fin . get () ; while (! fin . eof () ){ if ( c >= \'\' && c <= \' z \') // checks if c stores a standard char .{ temp += c ;} c = fin . get () ;} cout << temp ;/ / PROBLEM : No text is printed to the screen from the above command . cout << temp . size () ;/ / prints }}'

In [25]:
#text tokenizin - dataframe update
postsdf['TText'] = postsdf['PText'].apply(customtokenizer,args=(False, ))
postsdf['TText_NEG'] = postsdf['PText'].apply(customtokenizer,args=(True, ))
postsdf['TCode'] = postsdf['PCode'].apply(codetokenizer)
postsdf['TTitle'] = postsdf['Title'].apply(customtokenizer,args=(False, ))
postsdf['TTitle_NEG'] = postsdf['Title'].apply(customtokenizer,args=(True, ))
postsdf[['TText', 'TText_NEG', 'PText', 'TCode', 'PCode', 'TTitle', 'TTitle_NEG', 'Title']].head(20)

,TText,TText_NEG,PText,TCode,PCode,TTitle,TTitle_NEG,Title
Id,,,,,,,,
48046688,hi tri instal python packag call pysptk ubuntu...,hi tri instal python packag call pysptk ubuntu...,Hi I was trying to install a python package c...,pip install pysptk / usr / bin / ld : cannot f...,pip install pysptk\n /usr/bin/ld: cannot find ...,tri instal pythonn packag encount `` find -lgc...,tri instal pythonn packag encount `` find_NEG ...,tried to install a pythonn package but encount...
48046690,tri make chrome extens display differ text pop...,tri make chrome extens display differ text pop...,I am trying to make a chrome extension that d...,"< script src ="" popup . js ""> </ script >< htm...","<script src=""popup.js""></script>\n<html>\n<hea...",updat chrome extens popup text,updat chrome extens popup text,Update chrome extension popup text
48046378,call dll python use ctype want pass ctype arra...,call dll python use ctype want pass ctype arra...,"Calling a dll from Python using ctypes, I wan...",ReturnVec = ctypes . c_float * len ( arrA ) t ...,ReturnVec = ctypes.c_float * len(arrA)\nt = ty...,python ctype pass ctype array dll,python ctype pass ctype array dll,Python ctypes: how to pass ctypes array to DLL?
48046379,django app user creat video collag use multipl...,django app user creat video collag use multipl...,I have a Django app with which users can crea...,client_max_body_size 100M fastcgi_buffers 16k ...,client_max_body_size 100M\n fastcgi_buffers 8 ...,nginx gunicorn bad gateway upstream prematur c...,nginx gunicorn bad gateway upstream prematur c...,nginx gunicorn 502 bad gateway: upstream prema...
48046382,'m tri implement hittest use arkit line inters...,'m tri implement hittest use arkit line inters...,I'm trying to implement hittest that is used ...,"std :: pair < bool , double > linePlaneInterse...","std::pair<bool, double> linePlaneIntersection...",implement hit test,implement hit test,Implementing hit test
48046691,tri integr valid text entri field reduc code s...,tri integr valid text entri field reduc code s...,I have been trying to integrate some validati...,import tkinter as tk # Create root window root...,import tkinter as tk\n\n\n# Create root window...,struggl understand tkinter entri valid code,struggl understand tkinter entri valid code,Struggling to understand this Tkinter entry va...
48046692,get follow error tri remov user ssas server ad...,get follow error tri remov user ssas server ad...,Getting following error when trying to remove...,,,analysi servic – error tri remov user server a...,analysi servic – error tri remov user server a...,Analysis Services – Errors when trying to remo...
48046694,wonder way cach arbitrari data web request ont...,wonder way cach arbitrari data web request ont...,I am wondering if there is a way to cache arb...,SharedPreferences,SharedPreferences SharedPreferences SharedPref...,android cach disk,android cach disk,Android Caching To Disk
48046383,respons look like `` id '' `` name '' `` test ...,respons look like `` id '' `` name '' `` test ...,"My response looks like this: [\n {\n ""id""...",,,valid respons valu contain number sign,valid respons valu contain number sign,How to validate response value containing the ...


In [29]:
#Tokenized text saving
postsdf[['TText', 'TText_NEG', 'PText', 'TCode', 'PCode', 'TTitle', 'TTitle_NEG', 'Title', 'TagsSet', 'Tags']].to_csv('Cleaned_Posts.csv')
#postsdf.columns.values

In [26]:
#Text vectorizer feature
def cvect(corpus, max_df = 0.8, max_features=1000000, ngram_range = (1,1), min_df = 3):
    cvec = CountVectorizer(max_df=max_df, min_df=min_df, max_features=max_features, ngram_range = ngram_range, vocabulary=None, tokenizer=None)
    term_freq = cvec.fit_transform(corpus)
    zipped = zip([k for k in sorted(cvec.vocabulary_, key=cvec.vocabulary_.get, reverse=False)], np.asarray(term_freq.sum(axis=0))[0])
    vocabulary = [x for x in zipped]
    vocabulary_index = cvec.vocabulary_
    return term_freq, vocabulary, vocabulary_index

In [27]:
Posts_freq, Post_vocabulary, vocabulary_index = cvect(postsdf['TText'], max_df = .8, min_df = 1, max_features=5000000, ngram_range = (2,2))

In [28]:
articles_tfidf = TfidfTransformer().fit_transform(Posts_freq)
articles_tfidf.shape

(50000, 934472)

In [29]:
print(len(Post_vocabulary))
Post_vocabulary

934472


[('00 00', 277),
 ('00 000', 16),
 ('00 0000', 4),
 ('00 0000000', 1),
 ('00 0000028', 1),
 ('00 0000278', 1),
 ('00 000z', 12),
 ('00 0072261', 1),
 ('00 0072567', 1),
 ('00 00am', 3),
 ('00 00pm', 1),
 ('00 00z', 3),
 ('00 01', 23),
 ('00 0100', 2),
 ('00 012', 1),
 ('00 01dec2017', 1),
 ('00 02', 53),
 ('00 03', 30),
 ('00 0300', 1),
 ('00 035422', 2),
 ('00 04', 25),
 ('00 05', 23),
 ('00 06', 26),
 ('00 0644890', 1),
 ('00 065', 1),
 ('00 07', 9),
 ('00 075', 1),
 ('00 078', 1),
 ('00 079', 1),
 ('00 08', 16),
 ('00 082', 1),
 ('00 086', 1),
 ('00 09', 10),
 ('00 098', 1),
 ('00 10', 13),
 ('00 100', 1),
 ('00 1000', 1),
 ('00 101', 1),
 ('00 103', 1),
 ('00 104', 1),
 ('00 106', 2),
 ('00 107', 2),
 ('00 108', 1),
 ('00 11', 7),
 ('00 111', 1),
 ('00 112', 1),
 ('00 113', 3),
 ('00 114', 1),
 ('00 116', 2),
 ('00 117', 1),
 ('00 118', 2),
 ('00 119', 2),
 ('00 12', 11),
 ('00 120', 1),
 ('00 121', 1),
 ('00 122', 2),
 ('00 123', 2),
 ('00 124', 3),
 ('00 126', 2),
 ('00 128', 2),

In [33]:
#Pickle preparation
picklecorpus = {'CodeAnalysis':CodeAnalysis, 'TextAnalysis':TextAnalysis, 'TitleAnalysis':TitleAnalysis}

In [34]:
#Saving the dict as a pickle
with open('corpusprepared.pkl', 'wb') as f:
    pickle.dump(picklecorpus, f, pickle.HIGHEST_PROTOCOL) 

In [35]:
#Dicts loading
def load_obj(name):
    with open('corpusprepared.pkl', 'rb') as f:
        return pickle.load(f)
picklecorpus = load_obj('picklecorpus')

picklecorpus['TextAnalysis'] = TextAnalysis
picklecorpus['TitleAnalysis'] = TitleAnalysis
picklecorpus['CodeAnalysis'] = CodeAnalysis